<a href="https://colab.research.google.com/github/palis-dev/jupyter-notebooks/blob/main/ai_rag_wikipedia_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies


In [ ]:
!pip install faiss-cpu sentence_transformers wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.4 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679

# OpenAI Client

In [ ]:
from openai import OpenAI
from threading import Lock

class OpenAIClient:
    """
    Singleton para gerenciar a instância do cliente OpenAI.
    """
    _instance = None
    _lock = Lock()

    def __new__(cls, api_key):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(OpenAIClient, cls).__new__(cls)
                cls._instance._client = OpenAI(api_key=api_key)
            return cls._instance

    def get_client(self):
        """
        Retorna a instância do cliente OpenAI.
        """
        return self._client

API_KEY = "***"

openai_client = OpenAIClient(API_KEY)


# Crawler

In [ ]:
import wikipedia

wikipedia.set_lang("pt")

def collect_articles(keywords, max_articles=1000):
    articles = {}
    for keyword in keywords:
        try:
            results = wikipedia.search(keyword, results=10)
            for title in results:
                if title not in articles:
                    try:
                        page = wikipedia.page(title)
                        articles[title] = page.content
                        if len(articles) >= max_articles:
                            return articles
                    except Exception as e:
                        print(f"Error retrieving '{title}': {e}")
        except Exception as e:
            print(f"Error searching for '{keyword}': {e}")
    return articles

keywords = ["história", "civilizações antigas", "história Brasil", "Primeira Revolução Industrial", "Império Romano"]
documents = collect_articles(keywords)
print(f"Collected {len(documents)} documents.")


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Error retrieving 'Revolução Industrial (desambiguação)': "Revolução Industrial (desambiguação)" may refer to: 
Primeira Revolução Industrial
Segunda Revolução Industrial
Terceira Revolução Industrial
Quarta Revolução Industrial
Error retrieving 'Era industrial': "Era industrial" may refer to: 
Revolução Industrial
Segunda Revolução Industrial
Terceira Revolução Industrial
Indústria 4.0
Capitalismo industrial
Indústria
Industrialização
Todas as páginas cujo título começa por "Era industrial"
Todas as páginas que tenham "Era industrial" no título
Busca por "era industrial"
Collected 44 documents.


# Embedding

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

titles = list(documents.keys())
contents = list(documents.values())

embeddings = model.encode(contents, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

# Database

In [ ]:
import faiss
import numpy as np
import pickle
emb_array = np.array(embeddings).astype('float32')
dim = emb_array.shape[1]

index = faiss.IndexFlatL2(dim)

index.add(emb_array)
print(f"Indexed {index.ntotal} vectors.")

faiss.write_index(index, "faiss_index_history.idx")


with open("titles_mapping.pkl", "wb") as f:
    pickle.dump(titles, f)


Indexed 44 vectors.


# Vector similarity - top_k

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

model = SentenceTransformer('all-MiniLM-L6-v2')

index = faiss.read_index("faiss_index_history.idx")

with open("titles_mapping.pkl", "rb") as f:
    titles = pickle.load(f)

def vector_search(prompt, k=5):

    prompt_embedding = model.encode([prompt])
    prompt_embedding = np.array(prompt_embedding).astype('float32')

    distances, indices = index.search(prompt_embedding, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        # Check if the index is valid
        if idx < len(titles):
            results.append((titles[idx], dist))

    return results

# ISSO AQUI É SÓ PRA MOSTRAR O RETORNO, O PROMPT QUE USO LA EMBAIXO É OUTRO
prompt_text = "Influence of the Industrial Revolution on modern society"
results = vector_search(prompt_text, k=5)

print("Top search results:")
for title, distance in results:
    print(f"Title: {title}, Distance: {distance}")


Top search results:
Title: Indústria, Distance: 1.118760585784912
Title: Revolução Industrial, Distance: 1.2894843816757202
Title: Segunda Revolução Industrial, Distance: 1.2894843816757202
Title: Quarta Revolução Industrial, Distance: 1.3306961059570312
Title: Indústria química, Distance: 1.4562702178955078


In [ ]:
import openai
import numpy as np

def check_fake_news(prompt, top_k=3):

    context_results = vector_search(prompt, k=top_k)

    context_str = ""
    for title, distance in context_results:
        context_str += f"Document: {title}\n"

    messages = [
        {"role": "system", "content": f"""
        Below is a set of content context documents and a news prompt.
        Based on the context, determine whether the news prompt is likely fake or not.
        Respond with 'Fake' if the news appears inconsistent with given facts or 'Not Fake' if it seems plausible.

        Context:
        {context_str}
        Prompt:
        {prompt}
        """},
    ]

    response = openai_client.get_client().chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=50,
        temperature=0.3,
    )

    answer = response.choices[0].message.content
    return answer

news_prompt = "who discovered Brazil was Dom Pedro 1"
result = check_fake_news(news_prompt, top_k=3)
print("Generative model output:", result)


Generative model output: Fake
